# Task 2

## 1. Outline the Analysis Steps:
- We’ll import libraries, load the data, and visualize it.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys
import warnings
# import tensorflow as tf
# from keras.layers import LSTM
current_dir = os.getcwd()
# Append the parent directory to sys.path
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
# ignore warrnings
warnings.filterwarnings('ignore')

In [2]:
from scripts.oil_price_data_extraction import WorldDataAnalysis

In [3]:
data_extracter =WorldDataAnalysis()

In [4]:
data_extracter.fetch_world_data()

Fetching world data...
Fetch complete. Data type: <class 'wbdata.client.DataFrame'>
World data fetched and filtered by date successfully.
        date  GDP Growth (%)  Inflation Rate (%)  Unemployment Rate (%)  \
0 2023-01-01        2.833509            5.635669               4.916572   
1 2022-01-01        3.238021            7.930929               5.260711   
2 2021-01-01        6.350090            3.475403               6.064446   
3 2020-01-01       -2.878323            1.905664               6.608805   
4 2019-01-01        2.675966            2.216776               5.598931   

  Exchange Rate (USD)  
0                None  
1                None  
2                None  
3                None  
4                None  
Data saved to 'world_data.csv'.
